In [43]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [44]:
df=pd.read_csv("churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [45]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [46]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [47]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [48]:
from sklearn.preprocessing import LabelEncoder
label_enco=LabelEncoder()
df['Gender']=label_enco.fit_transform(df['Gender'])

In [49]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [50]:
df['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [51]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
data=ohe.fit_transform(df[['Geography']]).toarray()
df1=pd.DataFrame(data, columns=['France', 'Germany', 'Spain'])

In [52]:
df=pd.concat([df,df1], axis=1)

df.drop('Geography', axis=1, inplace=True)

In [53]:
print(data)

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [54]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [55]:
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_enco, f)

with open('one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(ohe, f)



In [56]:
x=df.drop('Exited', axis=1)
y=df['Exited']

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [59]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

In [60]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [61]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [62]:
x_train.shape

(8000, 12)

In [63]:
# build ANN model
model = Sequential([
    # first hidden layer connected to input layer
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),#hidden layer 2
    Dense(1, activation='sigmoid'),#output layer



])


In [64]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
import tensorflow
opi=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [66]:
model.compile(optimizer=opi, loss='binary_crossentropy', metrics=['accuracy'])

In [67]:
model

In [68]:
# setup tensorboard callback
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [72]:
# set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [73]:
history=model.fit(
    x_train,y_train,validation_data=(x_test, y_test),epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 6s 22ms/step - loss: 0.3386 - accuracy: 0.8615 - val_loss: 0.3484 - val_accuracy: 0.8530
Epoch 2/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3361 - accuracy: 0.8621 - val_loss: 0.3491 - val_accuracy: 0.8585
Epoch 3/100
250/250 [==============================] - 3s 14ms/step - loss: 0.3308 - accuracy: 0.8668 - val_loss: 0.3498 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 4s 16ms/step - loss: 0.3346 - accuracy: 0.8629 - val_loss: 0.3400 - val_accuracy: 0.8540
Epoch 5/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3303 - accuracy: 0.8656 - val_loss: 0.3459 - val_accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3271 - accuracy: 0.8669 - val_loss: 0.3430 - val_accuracy: 0.8555
Epoch 7/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3272 - accuracy: 0.8639 - val_loss: 0.3419 - val_accuracy: 

In [74]:
model.save('model.h5')

h:\DLprj\DeeEnv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [75]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit20250613-153802

Reusing TensorBoard on port 6008 (pid 6216), started 0:16:08 ago. (Use '!kill 6216' to kill it.)

In [ ]:
# load the pickel files